# Import packages

In [1]:
import pandas as pd
import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

# Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [2]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


# Provide smiles string(s)
and copy into them into my_smiles variable. It's a list of strings (a Python list can be of any size), the smiles are extracted for the csv resulting form the step 2 numbering the molecules.

In [3]:
react_nb = pd.read_csv("../data_csv/fragments_0-7.csv", sep=',')

my_smiles = [react_nb["C0C7_num"][i] for i in range(len(react_nb))]
#my_smiles

# Initialize the molecules and generate conformations
We will use some for loops to loop over the smiles, only one conformation will be taken for each molecules in this work

In [4]:
mols = []
for smile in my_smiles[:10]:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1ccc2c(c1)cccc2
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1ccc2c(c1)ccc(c2)[Si](C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1ccc2c(c1)ccc(c2)N1CCCCC1
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1ccc2c(c1)cc(cc2)O
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: COc1cccc2c1cccc2
INFO

###### Draw some molecules

In [5]:
#mols[0].draw(2)

# Create gaussian job for each conformation

In [6]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


/Users/julesschleinitz/Library/Application Support/hermite/C11H10O_b7a8
C11H10O_b7a8_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C14H18OSi_95fa
C14H18OSi_95fa_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C16H19NO_f5e7
C16H19NO_f5e7_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C11H10O2_d7c7
C11H10O2_d7c7_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C11H10O_9eb4
C11H10O_9eb4_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C8H10O_fd16
C8H10O_fd16_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C11H17NO_9080
C11H17NO_9080_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C11H17NO_d167
C11H17NO_d167_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C8H10O2_8f9c
C8H10O2_8f9c_conf_0
/Users/julesschleinitz/Library/Application Support/hermite/C9H12O2_df4c
C9H12O2_df4c_conf_0


In [7]:
sm.get_job_stats(split_by_can=True)

status,created
can,
COc1ccc(cc1)C,1
COc1ccc(cc1)[C@@H](N(C)C)C,1
COc1ccc(cc1)[C@H](N(C)C)C,1
COc1ccc2c(c1)cc(cc2)O,1
COc1ccc2c(c1)ccc(c2)N1CCCCC1,1
COc1ccc2c(c1)ccc(c2)[Si](C)(C)C,1
COc1ccc2c(c1)cccc2,1
COc1cccc2c1cccc2,1
OCCc1ccc(cc1)OC,1


# Before submitting jobs load gaussian module manually in hermite !

In [33]:
jobs = sm.get_jobs(slurm_status.failed)

In [34]:
jobs

{'225b9271c82487eb8daf63e40233e35a': slurm_job(can='COc1ccc2c(c1)cccc2', conformation=0, max_num_conformers=1, tasks=('opt b3lyp/6-31G* scf=(xqc,tight)', 'freq b3lyp/6-31G* volume NMR pop=NPA density=current Geom=AllCheck Guess=Read', 'TD(NStates=10, Root=1) b3lyp/6-31G* volume pop=NPA density=current Geom=AllCheck Guess=Read'), config={'theory': 'b3lyp', 'light_basis_set': '6-31G*', 'heavy_basis_set': 'LANL2DZ', 'generic_basis_set': 'genecp', 'max_light_atomic_number': 36}, job_id='9171', directory='/Users/julesschleinitz/Library/Application Support/hermite/C11H10O_b7a8', base_name='C11H10O_b7a8_conf_0', status=<slurm_status.failed: 4>, n_submissions=1, n_success_tasks=0),
 '610093eef291ee4fa36bf5d1e29edf48': slurm_job(can='COc1ccc2c(c1)ccc(c2)[Si](C)(C)C', conformation=0, max_num_conformers=1, tasks=('opt b3lyp/6-31G* scf=(xqc,tight)', 'freq b3lyp/6-31G* volume NMR pop=NPA density=current Geom=AllCheck Guess=Read', 'TD(NStates=10, Root=1) b3lyp/6-31G* volume pop=NPA density=current G

In [20]:
job.base_name

'C11H10O_b7a8_conf_0'

In [40]:
str(job.status) == 'slurm_status.failed'

True

In [33]:
str(job.status)

'slurm_status.failed'

In [39]:
job.status

AttributeError: 'slurm_status' object has no attribute 'status'